**Detail above the folder created and csv file**\
**In the mysql_dynamic_analysis folder**
1. docker-compose.yaml --> this file is to run the docker
2. simulate_activity.py --> python script for normal and attack activity
3. mysql_configs --> is has the vulnerable and non vulnerable configuraton with vulvariant and nonvulvariant
4. mysql_logs --> it contains the logs from the python script running with each folder with vulvariant and vulvariant and inside each folder we have 2 subfolder as error.log and general.log
5. Rest are the csv dataset created from the logs folder, please refer to code for further details.


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89"
}

def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    label = 1 if "vulvariant" in source_folder.lower() else 0

    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                timestamp = extract_timestamp(line)
                cwe = detect_cwe(line)
                entries.append({
                    "timestamp": timestamp,
                    "source": source_folder,
                    "log_type": log_type,
                    "log_message": line,
                    "cwe": cwe,
                    "label": label
                })
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)

if __name__ == "__main__":
    # Set the logs directory path
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset from the logs
    df_dataset = build_dataset(logs_directory)

    # Save the DataFrame to CSV
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv"
    df_dataset.to_csv(output_csv, index=False)

    print(f"Dataset created with {len(df_dataset)} entries and saved to {output_csv}")


Dataset created with 7367904 entries and saved to /content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv")

# Print the number of rows and columns
print("Total Rows:", df.shape[0])
print("Total Columns:", df.shape[1])


<ipython-input-2-4846741b0b56>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv")


Total Rows: 7367904
Total Columns: 6


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv")

# Display the first 5 rows
print(df.head())


<ipython-input-4-052c100e6f43>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset.csv")


   timestamp        source log_type  \
0        NaN  vulvariant47    error   
1        NaN  vulvariant47    error   
2        NaN  vulvariant47    error   
3        NaN  vulvariant47    error   
4        NaN  vulvariant47    error   

                                         log_message  cwe  label  
0  2025-03-07T18:59:11.694005Z 0 [Warning] TIMEST...  NaN      1  
1  2025-03-07T18:59:21.067981Z 0 [Warning] InnoDB...  NaN      1  
2  2025-03-07T18:59:24.289415Z 0 [Warning] InnoDB...  NaN      1  
3  2025-03-07T18:59:24.904353Z 0 [Warning] No exi...  NaN      1  
4  2025-03-07T18:59:24.925120Z 0 [Warning] Gtid t...  NaN      1  


**In the above dynamic_analysis_dataset time stamp was not coming**

In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

# Mapping for CWE detection
cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89"
}

# Functions
def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    label = 1 if "vulvariant" in source_folder.lower() else 0

    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                timestamp = extract_timestamp(line)
                cwe = detect_cwe(line)
                entries.append({
                    "timestamp": timestamp,
                    "source": source_folder,
                    "log_type": log_type,
                    "log_message": line,
                    "cwe": cwe,
                    "label": label
                })
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)


if __name__ == "__main__":
    # Path
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset
    df_dataset = build_dataset(logs_directory)

    # Display info
    print("Total Rows:", df_dataset.shape[0])
    print("Total Columns:", df_dataset.shape[1])
    print("First 5 Rows:")
    print(df_dataset.head())

    # Save the DataFrame to a CSV file
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis.csv"
    df_dataset.to_csv(output_csv, index=False)
    print(f"Dataset saved to {output_csv}")


Total Rows: 7367904
Total Columns: 6
First 5 Rows:
                     timestamp        source log_type  \
0  2025-03-07T18:59:11.694005Z  vulvariant47    error   
1  2025-03-07T18:59:21.067981Z  vulvariant47    error   
2  2025-03-07T18:59:24.289415Z  vulvariant47    error   
3  2025-03-07T18:59:24.904353Z  vulvariant47    error   
4  2025-03-07T18:59:24.925120Z  vulvariant47    error   

                                         log_message cwe  label  
0  2025-03-07T18:59:11.694005Z 0 [Warning] TIMEST...          1  
1  2025-03-07T18:59:21.067981Z 0 [Warning] InnoDB...          1  
2  2025-03-07T18:59:24.289415Z 0 [Warning] InnoDB...          1  
3  2025-03-07T18:59:24.904353Z 0 [Warning] No exi...          1  
4  2025-03-07T18:59:24.925120Z 0 [Warning] Gtid t...          1  
Dataset saved to /content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis.csv


**In the above dynamic_analysis cwe mapping was not proper**

In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

# Mapping for CWE detection
cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89"
}

def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    label = 1 if "vulvariant" in source_folder.lower() else 0

    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                timestamp = extract_timestamp(line)
                if timestamp:
                    message = line[len(timestamp):].strip()
                else:
                    message = line
                cwe = detect_cwe(message)
                entries.append({
                    "timestamp": timestamp,
                    "source": source_folder,
                    "log_type": log_type,
                    "log_message": message,
                    "cwe": cwe,
                    "label": label
                })
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)

if __name__ == "__main__":
    # Path to the logs directory
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset
    df_dataset = build_dataset(logs_directory)

    # Display the first 5 rows for verification
    print("First 5 rows of dataset:")
    print(df_dataset.head())

    # Save the dataset as CSV
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv"
    df_dataset.to_csv(output_csv, index=False)
    print(f"Dataset created with {len(df_dataset)} entries and saved to {output_csv}")



First 5 rows of dataset:
                     timestamp        source log_type  \
0  2025-03-07T18:59:11.694005Z  vulvariant47    error   
1  2025-03-07T18:59:21.067981Z  vulvariant47    error   
2  2025-03-07T18:59:24.289415Z  vulvariant47    error   
3  2025-03-07T18:59:24.904353Z  vulvariant47    error   
4  2025-03-07T18:59:24.925120Z  vulvariant47    error   

                                         log_message cwe  label  
0  0 [Warning] TIMESTAMP with implicit DEFAULT va...          1  
1  0 [Warning] InnoDB: New log files created, LSN...          1  
2  0 [Warning] InnoDB: Creating foreign key const...          1  
3  0 [Warning] No existing UUID has been found, s...          1  
4  0 [Warning] Gtid table is not ready to be used...          1  
Dataset created with 7367904 entries and saved to /content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv")

# Display 5 random rows
print(df.sample(5))


<ipython-input-7-8a7852894702>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv")


                           timestamp           source log_type  \
2775740                          NaN     vulvariant27  general   
2482417                          NaN     vulvariant28  general   
2405964  2025-03-07T18:32:23.283907Z     vulvariant28  general   
575509                           NaN     vulvariant48  general   
6162402                          NaN  nonvulvariant28  general   

                                               log_message  cwe  label  
2775740                    ,(@time_zone_id, -464220000, 0)  NaN      1  
2482417                     ,(@time_zone_id, 688554000, 5)  NaN      1  
2405964  1 Query\tINSERT INTO help_keyword (help_keywor...  NaN      1  
575509                      ,(@time_zone_id, 104925600, 3)  NaN      1  
6162402                    ,(@time_zone_id, 1869669000, 1)  NaN      1  


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv")

# Display 5 random rows
print(df.sample(5))


<ipython-input-8-8a7852894702>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset1.csv")


                           timestamp           source log_type  \
1511734                          NaN     vulvariant36  general   
1672498                          NaN     vulvariant33  general   
2913596                          NaN     vulvariant26  general   
6740082  2025-03-07T17:51:46.335018Z   nonvulvariant1  general   
6886672                          NaN  nonvulvariant15  general   

                                               log_message  cwe  label  
1511734                     ,(@time_zone_id, 701798400, 2)  NaN      1  
1672498                    ,(@time_zone_id, -560203200, 2)  NaN      1  
2913596                     ,(@time_zone_id, 386699400, 1)  NaN      1  
6740082  3 Query\tINSERT INTO time_zone_transition_type...  NaN      1  
6886672                    ,(@time_zone_id, 1004230800, 8)  NaN      1  


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

# mapping for CWE detection
cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89",
    r"TIMESTAMP with implicit DEFAULT value is deprecated": "CWE-489",
    r"A deprecated TLS version TLSv1": "CWE-326",
    r"A deprecated TLS version TLSv1\.1": "CWE-326",
    r"CA certificate .* is self signed": "CWE-295",
    r"created with an empty password": "CWE-798",
    r"'NO_AUTO_CREATE_USER' sql mode was not set": "CWE-16"
}

def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    container_type = "vulnerable" if "vulvariant" in source_folder.lower() else "nonvulnerable"

    current_entry = None
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.rstrip("\n")
                if not line:
                    continue
                ts = extract_timestamp(line)
                if ts:
                    if current_entry is not None:
                        entries.append(current_entry)
                    remainder = line[len(ts):].strip()
                    current_entry = {
                        "timestamp": ts,
                        "source": source_folder,
                        "container_type": container_type,
                        "log_type": log_type,
                        "log_message": remainder,
                        "cwe": "",
                        "label": None
                    }
                else:
                    if current_entry is not None:
                        current_entry["log_message"] += " " + line.strip()
                    else:
                        continue
            if current_entry is not None:
                entries.append(current_entry)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
.
    for entry in entries:
        cwe_detected = detect_cwe(entry["log_message"])
        entry["cwe"] = cwe_detected if cwe_detected else "0"
        entry["label"] = 1 if cwe_detected else 0

    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)

if __name__ == "__main__":
    # Set the logs directory
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset
    df_dataset = build_dataset(logs_directory)

    # Print 5 random rows for verification
    print("Random 5 rows from dataset:")
    print(df_dataset.sample(5))

    # Save the DataFrame to a CSV file
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset2.csv"
    df_dataset.to_csv(output_csv, index=False)

    print(f"Dataset created with {len(df_dataset)} entries and saved to {output_csv}")


Random 5 rows from dataset:
                          timestamp           source container_type log_type  \
539414  2025-03-07T18:16:54.791704Z   nonvulvariant8     vulnerable  general   
271835  2025-03-07T18:33:51.742221Z     vulvariant24     vulnerable  general   
765455  2025-03-07T18:32:21.795097Z  nonvulvariant26     vulnerable  general   
214481  2025-03-07T18:39:28.294765Z     vulvariant32     vulnerable  general   
705729  2025-03-07T18:34:03.220894Z  nonvulvariant22     vulnerable  general   

                                              log_message cwe  label  
539414       3 Query\tSET @time_zone_id= LAST_INSERT_ID()   0      0  
271835  3 Query\tINSERT INTO time_zone_transition_type...   0      0  
765455  1 Query\tINSERT INTO help_keyword (help_keywor...   0      0  
214481                        1 Query\tDROP PREPARE stmt;   0      0  
705729  3 Query\tINSERT INTO time_zone_transition_type...   0      0  
Dataset created with 911888 entries and saved to /content/drive/M

In [ ]:
import pandas as pd

# Load the CSV file (adjust the path as needed)
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset2.csv")

# Display 5 random rows
print(df.sample(5))


                          timestamp           source container_type log_type  \
186233  2025-03-07T18:55:16.153106Z     vulvariant35     vulnerable  general   
457162  2025-03-07T18:28:11.804081Z     vulvariant17     vulnerable  general   
439724  2025-03-07T18:32:24.282778Z     vulvariant19     vulnerable  general   
562551  2025-03-07T17:59:45.680309Z   nonvulvariant9     vulnerable  general   
902834  2025-03-07T18:33:54.917728Z  nonvulvariant20     vulnerable  general   

                                              log_message cwe  label  
186233                             1 Query\tEXECUTE stmt;   0      0  
457162  3 Query\tINSERT INTO time_zone_transition (Tim...   0      0  
439724  1 Query\tINSERT INTO help_relation (help_topic...   0      0  
562551  1 Prepare\tCREATE TABLE performance_schema.thr...   0      0  
902834  3 Query\tINSERT INTO time_zone_transition (Tim...   0      0  


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset2.csv")

# Display 5 random rows
print(df.sample(5))


                          timestamp           source container_type log_type  \
254388  2025-03-07T18:33:30.377802Z     vulvariant23     vulnerable  general   
430003  2025-03-07T18:27:27.729342Z     vulvariant15     vulnerable  general   
190553  2025-03-07T18:55:58.982183Z     vulvariant35     vulnerable  general   
594782  2025-03-07T18:59:33.983834Z  nonvulvariant43     vulnerable  general   
640547  2025-03-07T18:59:40.376576Z  nonvulvariant47     vulnerable  general   

                                              log_message cwe  label  
254388  3 Query\tINSERT INTO time_zone_name (Name, Tim...   0      0  
430003                        1 Query\tDROP PREPARE stmt;   0      0  
190553  3 Query\tINSERT INTO time_zone_transition_type...   0      0  
594782                             1 Query\tEXECUTE stmt;   0      0  
640547  1 Query\tINSERT INTO help_relation (help_topic...   0      0  


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset2.csv")

# Display 5 random rows
print(df.sample(5))


                          timestamp        source container_type log_type  \
240487  2025-03-07T18:55:18.092065Z  vulvariant33     vulnerable  general   
13753   2025-03-07T18:16:19.307868Z   vulvariant6     vulnerable  general   
12035   2025-03-07T18:16:14.058928Z   vulvariant6     vulnerable  general   
338593  2025-03-07T17:51:54.723061Z   vulvariant3     vulnerable  general   
410238  2025-03-07T18:33:35.540636Z  vulvariant21     vulnerable  general   

                                              log_message cwe  label  
240487  1 Query\tINSERT INTO help_topic (help_topic_id...   0      0  
13753   1 Query\tINSERT INTO help_topic (help_topic_id...   0      0  
12035                         1 Query\tDROP PREPARE stmt;   0      0  
338593       3 Query\tSET @time_zone_id= LAST_INSERT_ID()   0      0  
410238  3 Query\tINSERT INTO time_zone (Use_leap_secon...   0      0  


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

# mapping for CWE detection
cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89",
    r"TIMESTAMP with implicit DEFAULT value is deprecated": "CWE-489",
    r"A deprecated TLS version TLSv1": "CWE-326",
    r"A deprecated TLS version TLSv1\.1": "CWE-326",
    r"CA certificate .* is self signed": "CWE-295",
    r"created with an empty password": "CWE-798",
    r"'NO_AUTO_CREATE_USER' sql mode was not set": "CWE-16"
}

def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    container_type = "vulnerable" if "vulvariant" in source_folder.lower() else "nonvulnerable"

    current_entry = None
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.rstrip("\n")
                if not line:
                    continue
                ts = extract_timestamp(line)
                if ts:
                    if current_entry is not None:
                        entries.append(current_entry)
                    remainder = line[len(ts):].strip()
                    current_entry = {
                        "timestamp": ts,
                        "source": source_folder,
                        "container_type": container_type,
                        "log_type": log_type,
                        "log_message": remainder,
                        "cwe": "",
                        "label": None
                    }
                else:
                    if current_entry is not None:
                        current_entry["log_message"] += " " + line.strip()
            if current_entry is not None:
                entries.append(current_entry)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    for entry in entries:
        cwe_detected = detect_cwe(entry["log_message"])
        entry["cwe"] = cwe_detected if cwe_detected else "0"
        if container_type == "vulnerable":
            entry["label"] = 1 if cwe_detected else 0
        else:
            entry["label"] = 0
    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)

if __name__ == "__main__":
    # Set the logs directory
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset
    df_dataset = build_dataset(logs_directory)

    # Print 5 random rows for verification
    print("Random 5 rows from dataset:")
    print(df_dataset.sample(5))

    # Save the dataset to CSV
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset3.csv"
    df_dataset.to_csv(output_csv, index=False)

    print(f"Dataset created with {len(df_dataset)} entries and saved to {output_csv}")


Random 5 rows from dataset:
                          timestamp           source container_type log_type  \
421174  2025-03-07T18:33:13.974277Z     vulvariant20     vulnerable  general   
814140  2025-03-07T18:39:32.748462Z  nonvulvariant33     vulnerable  general   
694153  2025-03-07T18:32:11.880499Z  nonvulvariant22     vulnerable  general   
554602  2025-03-07T18:16:49.268143Z   nonvulvariant7     vulnerable  general   
864879  2025-03-07T18:27:37.251176Z  nonvulvariant11     vulnerable  general   

                                              log_message cwe  label  
421174  3 Query\tINSERT INTO time_zone_transition (Tim...   0      0  
814140  1 Query\tINSERT INTO help_keyword (help_keywor...   0      0  
694153                        1 Query\tDROP PREPARE stmt;   0      0  
554602  3 Query\tINSERT INTO time_zone (Use_leap_secon...   0      0  
864879  1 Query\tINSERT INTO help_relation (help_topic...   0      0  
Dataset created with 911888 entries and saved to /content/drive/M

In [ ]:
import pandas as pd

# Load your CSV dataset
df = pd.read_csv("/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/dynamic_analysis_dataset3.csv")

# Count occurrences of each label
label_counts = df['label'].value_counts()

print("Label Counts:")
print(label_counts)


Label Counts:
label
0    909590
1      2298
Name: count, dtype: int64


**without proper mapping just put 1 for all non vulnerable config which is not correct**

In [ ]:
import os
import re
import pandas as pd
from datetime import datetime

# mapping for CWE detection
cwe_mapping = {
    r"DROP DATABASE": "CWE-287",
    r"GRANT ALL PRIVILEGES": "CWE-264",
    r"SELECT .*SLEEP": "CWE-400",
    r"non_existing_table' OR '1'='1": "CWE-89",
    r"TIMESTAMP with implicit DEFAULT value is deprecated": "CWE-489",
    r"A deprecated TLS version TLSv1": "CWE-326",
    r"A deprecated TLS version TLSv1\.1": "CWE-326",
    r"CA certificate .* is self signed": "CWE-295",
    r"created with an empty password": "CWE-798",
    r"'NO_AUTO_CREATE_USER' sql mode was not set": "CWE-16"
}

def extract_timestamp(line):
    pattern = r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)'
    match = re.match(pattern, line)
    if match:
        return match.group(1)
    return ""

def determine_log_type(file_name):
    lower_name = file_name.lower()
    if "error" in lower_name:
        return "error"
    elif "general" in lower_name:
        return "general"
    else:
        return "unknown"

def detect_cwe(log_message):
    for pattern, cwe in cwe_mapping.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return cwe
    return ""

def determine_container_type(source_folder):
    s = source_folder.lower()
    if s.startswith("nonvulvariant"):
        return "nonvulnerable"
    elif s.startswith("vulvariant"):
        return "vulnerable"
    else:
        return "unknown"

def parse_log_file(file_path, source_folder):
    entries = []
    log_type = determine_log_type(os.path.basename(file_path))
    container_type = determine_container_type(source_folder)

    current_entry = None
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.rstrip("\n")
                if not line:
                    continue
                ts = extract_timestamp(line)
                if ts:
                    if current_entry is not None:
                        entries.append(current_entry)
                    remainder = line[len(ts):].strip()
                    current_entry = {
                        "timestamp": ts,
                        "source": source_folder,
                        "container_type": container_type,
                        "log_type": log_type,
                        "log_message": remainder,
                        "cwe": "",
                        "label": None
                    }
                else:
                    if current_entry is not None:
                        current_entry["log_message"] += " " + line.strip()
            if current_entry is not None:
                entries.append(current_entry)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    # Determine CWE and set label for each entry.
    for entry in entries:
        cwe_detected = detect_cwe(entry["log_message"])
        entry["cwe"] = cwe_detected if cwe_detected else "0"
        if container_type == "vulnerable":
            entry["label"] = 1
        else:
            entry["label"] = 0
    return entries

def build_dataset(logs_root):
    all_entries = []
    for variant in os.listdir(logs_root):
        variant_path = os.path.join(logs_root, variant)
        if os.path.isdir(variant_path):
            for file in os.listdir(variant_path):
                file_path = os.path.join(variant_path, file)
                entries = parse_log_file(file_path, variant)
                all_entries.extend(entries)
    return pd.DataFrame(all_entries)

if __name__ == "__main__":
    # Set the logs directory (update to your actual Drive path)
    logs_directory = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/mysql_logs"

    # Build the dataset from the log files
    df_dataset = build_dataset(logs_directory)

    # Print 5 random rows for verification
    print("Random 5 rows from dataset:")
    print(df_dataset.sample(5))

    # Count the number of vulnerable (label=1) and non-vulnerable (label=0) entries
    label_counts = df_dataset['label'].value_counts()
    print("\nLabel counts:")
    print(label_counts)

    # Save the dataset to a CSV file
    output_csv = "/content/drive/MyDrive/Dynamic_Analysis/mysql_dynamic_analysis/ffdynamic_analysis_dataset.csv"
    df_dataset.to_csv(output_csv, index=False)

    print(f"\nDataset created with {len(df_dataset)} entries and saved to {output_csv}")


Random 5 rows from dataset:
                          timestamp           source container_type log_type  \
657145  2025-03-07T17:51:04.657507Z   nonvulvariant4  nonvulnerable  general   
88410   2025-03-07T17:52:09.098174Z      vulvariant5     vulnerable  general   
716154  2025-03-07T18:33:31.923557Z  nonvulvariant25  nonvulnerable  general   
18015   2025-03-07T18:16:43.052599Z      vulvariant6     vulnerable  general   
610912  2025-03-07T18:59:31.607723Z  nonvulvariant45  nonvulnerable  general   

                                              log_message cwe  label  
657145  1 Query\tINSERT INTO help_relation (help_topic...   0      0  
88410   3 Query\tINSERT INTO time_zone_transition_type...   0      1  
716154  3 Query\tINSERT INTO time_zone_name (Name, Tim...   0      0  
18015   3 Query\tINSERT INTO time_zone (Use_leap_secon...   0      1  
610912                        1 Query\tDROP PREPARE stmt;   0      0  

Label counts:
label
1    527457
0    384431
Name: count, dtype: 